# Setup

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Number of custom components

In [ ]:
with Apps.init() as apps:
    apps.where(lambda app: app.components.some(lambda component: component.type == "Custom")).select(
        {
            "Unique custom components": lambda app: app.components.filter(lambda component: component.type == "Custom")
            .map(lambda component: component[".tagName"])
            .filter(lambda tagName: tagName != None)
            .unique()
            .length
        }
    ).order_by(lambda app: app["Unique custom components"], reverse=True).table()

## Custom components per app

In [ ]:
with Apps.init() as apps:
    apps.where(lambda app: app.components.some(lambda component: component.type == "Custom")).select(
        {
            "Custom components": lambda app: app.components.filter(lambda component: component.type == "Custom").map(
                lambda component: component
            )
        }
    ).order_by(lambda app: (app.org, app.app)).table()

## Apps using Subform in prod

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod" and app.components.some(lambda component: component.type == "Subform")
    ).table()

## DataModelBindings in List-component

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod" and app.components.some(lambda component: component.type == "List")
    ).select(
        {
            "List bindings": lambda app: app.components.filter(lambda component: component.type == "List")
            .flat_map(lambda component: component[".dataModelBindings | keys"])
            .unique()
            .sort()
        }
    ).table()

# Using multi-page

In [ ]:
import re

multi_page_pattern = re.compile(r"^\d+:.")

with Apps.init() as apps:
    apps.select(
        {
            "Components with multi-page": lambda app: app.components.filter(
                lambda component: component.type in ["Group", "RepeatingGroup"]
                and component[".children[]?", :].some(
                    lambda childId: re.match(multi_page_pattern, str(childId)) != None
                )
            )
        }
    ).where(lambda app: app["Components with multi-page"].length > 0).table()

## Dropdown or Multiselect using autocomplete

In [ ]:
with Apps.init() as apps:
    apps.select(
        {
            "Autocomplete": lambda app: app.components.filter(
                lambda component: component.type in ["MultipleSelect", "Dropdown"]
                and component[".autocomplete"] != None
            )
        }
    ).where(lambda app: app["Autocomplete"].length > 0).table()

## Apps using SimpleTable

In [ ]:
with Apps.init() as apps:
    apps.where(lambda app: app.components.some(lambda component: component.type == "SimpleTable")).table()